In [1]:
# Install exta packages

!pip install --upgrade pip
!pip install smart-open
!pip install flatten_json

In [2]:
# Import libraries and set up variables

# import the libraries that we will use
import pandas as pd
import numpy as np
from smart_open import open
from flatten_json import flatten
from datetime import datetime

# s3 locations
bucket = "emba-final-project"
in_folder = "training-data"
out_folder = "csv"

# input files
in_business_file = "business_test.json"
in_photos_file = "photos_test.json"
in_review_file = "review_test.json"
in_user_file = "user_test.json"

# output files
out_business_file = "business_test.csv"
out_photos_file = "photos_test.csv"
out_review_file = "review_test.csv"
out_user_file = "user_test.csv"

# Comment out the next two groups to use small test files
# Uncomment the next two groups to use large training files

# input files
in_business_file = "business_train.json"
in_photos_file = "photos_train.json"
in_review_file = "review_train.json"
in_user_file = "user_train.json"

# output files
out_business_file = "business_test.csv"
out_photos_file = "photos_test.csv"
out_review_file = "review_test.csv"
out_user_file = "user_test.csv"

# input paths
in_business_path = f's3://{bucket}/{in_folder}/{in_business_file}'
in_photos_path = f's3://{bucket}/{in_folder}/{in_photos_file}'
in_review_path = f's3://{bucket}/{in_folder}/{in_review_file}'
in_user_path = f's3://{bucket}/{in_folder}/{in_user_file}'

# output paths
out_business_path = f's3://{bucket}/{out_folder}/{out_business_file}'
out_photos_path = f's3://{bucket}/{out_folder}/{out_photos_file}'
out_review_path = f's3://{bucket}/{out_folder}/{out_review_file}'
out_user_path = f's3://{bucket}/{out_folder}/{out_user_file}'


In [3]:
# Define a few functions

# Add a comma to the string passed in if the last character is not a '{'
# i.e. add a comma if we are not starting a new entry in the dict
def add_comma_if_needed(dict_str):
    if not dict_str.endswith('{'):
        dict_str = dict_str + ','
    return dict_str


##############################


# Define function to flatten yelp academic JSON data set
def fix_dict(data, list_of_lists = [], dict_of_operations = {}):
    new_dict_str = "{"
    
    # Create list of keys from dict_of_operations
    list_of_dos = []
    for do_key, do_value in dict_of_operations.items():
        list_of_dos.append(do_key)
    
    # Process and fix the JSON string data
    for key, value in data.items():

        # Check key. If its in the list to operate on its values then:
        # step 1) get the operation from dict_of_operations
        # step 2) do the operation on the values and use the result
        #         as a new value for the key
        if key in list_of_dos:
            new_dict_str = add_comma_if_needed(new_dict_str)
            # Get the operation from the dictionary
            operation = dict_of_operations.get(key)
            # Initialize new_value
            new_value = ""
            # Convert value to list or dict for operation
            # Check if string is really a quoted dictionary
            if value.startswith('{') and value.endswith('}'):
                pass
            # Else make it into a list
            else:
                # Create list from comma delimited string and remove
                # leading and trailing white space
                new_list = [x.strip() for x in value.split(',')]
            # Find and do the operation
            if operation == "count":
                # Count the items shown in the key's value
                new_value = len(new_list)
                # If there is only one item and it is "None" change count to zero
                if new_value == 1 and (new_list[0] == "None" or new_list[0] == ""):
                    new_value = 0
            elif operation == "sum":
                # Add the items shown in the key's value
                new_value = sum(new_list)
            else:
                pass
            new_dict_str = new_dict_str + '"' + str(key) + '":' + str(new_value)
            # Go to the next key since we just did special processing for this key
            continue

        # Check key. If its in the list to convert its values to:
        # step 1) a list
        # step 2) a dict of booleans with value True
        if key in list_of_lists:
            new_dict_str = add_comma_if_needed(new_dict_str)
            # Create list from comma delimited string and remove
            # leading and trailing white space
            new_list = [x.strip() for x in value.split(',')]
            # print(new_list)
            # Start a new dictionary with the key that got us here
            new_dict_str = new_dict_str + '"' + str(key) + '":{'
            # Iterate over the list and add each item as a key
            # with a value of True
            for i in new_list:
                new_dict_str = add_comma_if_needed(new_dict_str)
                new_dict_str = new_dict_str + '"' + str(i) + '":' + str(True)
            # end the dictionary
            new_dict_str = new_dict_str + '}'
            # Go to the next key since we just did special processing for this key
            continue

        # Add int and float entries as is without qoutes
        if isinstance(value, (int, float)):
            new_dict_str = add_comma_if_needed(new_dict_str)
            new_dict_str = new_dict_str + '"' + str(key) + '":' + str(value)

        # Fix any nested dictionary entries and add them
        if isinstance(value, (dict)):
            new_dict_str = add_comma_if_needed(new_dict_str)
            new_dict_str = new_dict_str + '"' + str(key) + '":' + fix_dict(value)

        # Add list entries as is without quotes
        # (kept this separate from int and float in case future special processing is needed)
        if isinstance(value, (list)):
            new_dict_str = add_comma_if_needed(new_dict_str)
            new_dict_str = new_dict_str + '"' + str(key) + '":' + str(value)

        # Some strings may actually be a dictionary in quotes
        # If it's a quoted dictionary, remove the quotes, fix, and keep it nested in place
        # Otherwise, just add the string as is
        if isinstance(value, (str)):
            new_dict_str = add_comma_if_needed(new_dict_str)
            # Check if string is really a quoted dictionary
            if value.startswith('{') and value.endswith('}'):
                value = eval(value)
                new_dict_str = new_dict_str + '"' + str(key) + '":' + fix_dict(value)
            # if not, just add it as is
            else:
                new_dict_str = new_dict_str + '"' + str(key) + '":"' + str(value) + '"'

    # Close the dictionary 
    new_dict_str = new_dict_str + '}'
    # and return fixed dictionary (as a string--it gets converted to a dict object upon return)
    return new_dict_str


##############################


def json_string_to_flat_df(line, special_keys = [], operation_keys = {}):

    # Replace "null" strings with empty strings
    line = line.replace("null", "\"\"") # make null look like ""
    # Remove backslashes that occur at the end of a quoted string
    line = line.replace("\\\\\",\"", "\",\"") #  make \\"," look like ","
    line = line.replace("\\\",\"", "\",\"") # make \"," look like ","
    # Replace escaped double quotes with a single quote
    line = line.replace("\\\"", "'") # make \" look like '
    # Escape any backslashes that are left with another backslash
    line = line.replace("\\", "\\\\") # make \ look like \\
    
    # print(line)
    
    # Change string to a dictionary object
    ydict = eval(line)
    # print(ydict)
    
    # Fix stingified dictionaries and change big string of values into
    # into dictionary of booleans
    better_ydict = eval(fix_dict(ydict, special_keys, operation_keys))
    
    # Flatten the dictionary to be added to a DataFrame 
    ydict_flattened = pd.json_normalize(better_ydict)
    
    return ydict_flattened


In [4]:
# Process business data

# Initialize DataFrame
bdf = pd.DataFrame([])

# Read in the data a line at a time
for line in open(in_business_path, encoding='utf8'):
    # print(line)
    
    # Flatten and fix json string to be added to DataFram
    flat_df = json_string_to_flat_df(line, ['categories'])
    
    # Add the flattened dictionary to a DataFrame
    bdf = bdf.append(flat_df, ignore_index=True)

    
# Eliminate unwanted columns
bdf.drop( \
         ['address', \
          'latitude', \
          'longitude', \
          'is_open', \
          'hours', \
          'hours.Monday', \
          'hours.Tuesday', \
          'hours.Wednesday', \
          'hours.Thursday', \
          'hours.Friday', \
          'hours.Saturday', \
          'hours.Sunday'], \
         axis=1, \
         inplace=True \
        )    

# Save the DataFrame to CSV files, both locally (for easy inspection)
# and to an S3 bucket
bdf.to_csv(out_business_file) # local
bdf.to_csv(out_business_path) # S3

# Review DataFrame
bdf.head(5)

,business_id,name,city,state,postal_code,stars,review_count,attributes.RestaurantsTableService,attributes.WiFi,attributes.BikeParking,...,categories.Vintage & Consignment,categories.Shopping,categories.Furniture Stores,categories.Home & Garden,categories.Beauty & Spas,categories.Hair Salons,categories.Gyms,categories.Active Life,categories.Interval Training Gyms,categories.Fitness & Instruction
0,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,Boulder,CO,80302,4.0,86,True,u'free',True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,Portland,OR,97218,4.0,126,NaN,u'free',False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bvN78flM8NLprQ1a1y5dRg,The Reclaimory,Portland,OR,97214,4.5,13,NaN,NaN,False,...,True,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN
3,oaepsyvc0J17qwi8cfrOWg,Great Clips,Orange City,FL,32763,3.0,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,True,True,NaN,NaN,NaN,NaN
4,PE9uqAjdw0E4-8mjGl3wVA,Crossfit Terminus,Atlanta,GA,30316,4.0,14,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,True,True,True,True


In [5]:
# Process photo data

# Initialize DataFrame
pdf = pd.DataFrame([])

# Read in the data a line at a time
for line in open(in_photos_path, encoding='utf8'):
    # print(line)
    
    # Flatten and fix json string to be added to DataFram
    flat_df = json_string_to_flat_df(line)
    
    # Add the flattened dictionary to a DataFrame
    pdf = pdf.append(flat_df, ignore_index=True)


# Save the DataFrame to CSV files, both locally (for easy inspection)
# and to an S3 bucket
pdf.to_csv(out_photos_file) # local
pdf.to_csv(out_photos_path) # S3

# Review DataFrame
pdf.head(5)

,photo_id,business_id,caption,label
0,V1D1gaJLZfQmbUK47yWucw,6iYb2HFDywm3zjuRg0shjw,The Hotbox Coffee Porter is amazing!,drink
1,JJcbBzWxk1TTNs5XIUP8Ag,tCbdrRPZA0oiIYSmHG3J0w,"Almond croissant\, made fresh in our bakery ev...",food
2,bamuoAQPHguDmFvl_BkoOg,tCbdrRPZA0oiIYSmHG3J0w,,interior
3,0Hko_ElNxdGSYyC7S6xB2g,tCbdrRPZA0oiIYSmHG3J0w,,interior
4,HUVBN5pqRE6nPYLadGCNZQ,6iYb2HFDywm3zjuRg0shjw,Beers on tap. Really good selection and great ...,interior


In [6]:
# Process review data

# Initialize DataFrame
rdf = pd.DataFrame([])

# Read in the data a line at a time
for line in open(in_review_path, encoding='utf8'):
    # print(line)
    
    # Flatten and fix json string to be added to DataFram
    flat_df = json_string_to_flat_df(line)
    
    # Add the flattened dictionary to a DataFrame
    rdf = rdf.append(flat_df, ignore_index=True)


# Add individual date and time features from existing datetime feature
now = datetime.now()
rdf['dt_obj'] = pd.to_datetime(rdf['date'], format='%Y-%m-%d %H:%M:%S')
rdf['year'] = rdf['dt_obj'].dt.year
rdf['month'] = rdf['dt_obj'].dt.month
rdf['hour'] = rdf['dt_obj'].dt.hour
rdf['review_age'] = now - rdf.dt_obj
rdf['review_age'] = rdf.review_age / np.timedelta64(1, 'Y')


# Save the DataFrame to CSV files, both locally (for easy inspection)
# and to an S3 bucket
rdf.to_csv(out_review_file) # local
rdf.to_csv(out_review_path) # S3

# Review DataFrame
rdf.head(5)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,dt_obj,year,month,hour,review_age
0,NYyoUk7FL6D4EWYiZfzrMw,qU0BZ7yIl8s9ocBy4enISw,G1KjtSHx7L-a_G-TQWEnoA,4.0,5,2,0,Yelp came through again! Was in Orlando for th...,2011-07-04 16:13:22,2011-07-04 16:13:22,2011,7,16,9.746932
1,ROIVLs-_MjKcir9yL9b3yQ,61DrjFcfN1-bS7MrmNA7Jg,LPa3rpKsApYeiQL_qxZ_jA,3.0,0,0,0,Eh . . . we wound up here on a whim a few year...,2011-06-24 03:38:20,2011-06-24 03:38:20,2011,6,3,9.775747
2,sPWRG7i-gwJjo0nDPr87Dw,24gb1QBBEl2xHtVeTzDsCQ,tCbdrRPZA0oiIYSmHG3J0w,4.0,1,0,1,Elephant's contacted me the same day I posted ...,2012-07-16 05:04:05,2012-07-16 05:04:05,2012,7,5,8.713276
3,w3ge0N2w88RY41-0r7zmcw,7yjvnYDms_mrmBJ75RZlnw,oaepsyvc0J17qwi8cfrOWg,1.0,0,0,0,"Worst hair cut ever, not only myself but my hu...",2016-10-26 15:23:19,2016-10-26 15:23:19,2016,10,15,4.432750
4,I0q_GX7IkjecNdr4lQDzcQ,AsDPajAdVBsXydg60AIsdw,tCbdrRPZA0oiIYSmHG3J0w,5.0,0,0,0,I'm not usually a fan of airport food. I usual...,2015-04-28 21:11:10,2015-04-28 21:11:10,2015,4,21,5.929723


In [7]:
# Process user data

# Initialize DataFrame
udf = pd.DataFrame([])

# Read in the data a line at a time
for line in open(in_user_path, encoding='utf8'):
    # print(line)
    
    # Flatten and fix json string to be added to DataFram
    flat_df = json_string_to_flat_df(line, [], {"friends":"count","elite":"count"})
    
    # Add the flattened dictionary to a DataFrame
    udf = udf.append(flat_df, ignore_index=True)


# Add individual date and time features from existing datetime feature
now = datetime.now()
udf['dt_obj'] = pd.to_datetime(udf['yelping_since'], format='%Y-%m-%d %H:%M:%S')
udf['yelp_age'] = now - udf.dt_obj
udf['yelp_age'] = udf.yelp_age / np.timedelta64(1, 'Y')


# Save the DataFrame to CSV files, both locally (for easy inspection)
# and to an S3 bucket
udf.to_csv(out_user_file) # local
udf.to_csv(out_user_path) # S3

# Review DataFrame
udf.head(5)

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos,dt_obj,yelp_age
0,24gb1QBBEl2xHtVeTzDsCQ,Lisa,94,2011-01-29 01:48:13,278,58,83,1,352,6,...,0,1,1,10,5,5,4,0,2011-01-29 01:48:13,10.175690
1,Eypq5gLLjCapBVVnMw_MyA,Misha,4880,2007-10-29 08:53:07,4199,2190,3018,14,807,194,...,9,4,54,93,124,124,63,11,2007-10-29 08:53:07,13.427516
2,zoCmtXdUO8Mk9NAYqJCheg,Brad,27,2009-07-01 19:17:38,32,7,2,0,99,0,...,0,0,0,0,3,3,1,0,2009-07-01 19:17:38,11.753467
3,AsDPajAdVBsXydg60AIsdw,Stephanie,26,2012-07-26 17:27:46,38,4,35,0,1,0,...,0,0,0,0,0,0,0,0,2012-07-26 17:27:46,8.684483
4,7yjvnYDms_mrmBJ75RZlnw,Lana,2,2016-04-02 04:10:12,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2016-04-02 04:10:12,5.000776
